In [20]:
import pandas as pd
import sys
import tensorflow as tf

from keras.models import load_model
from deepkt import DKTModel

sys.path.append('/home/grenouille/Documents/jenga/final_project/code/kuze_dkt_imp')

In [2]:
model = load_model('student_prediction', custom_objects={'DKTModel': DKTModel, 'custom_loss': DKTModel.custom_loss})

In [3]:
model

In [5]:
data = pd.read_csv('data/kuze_data/predictor_evaluations.csv')

In [6]:
data.shape

(1515, 45)

In [22]:
taxonomies = pd.read_csv('data/kuze_data/factorized_math_taxonomies.csv')

In [23]:
data['factorized_taxonomy_code'] = data['taxonomy_id_0'].map(
    taxonomies.set_index('taxonomy_id_0')['factorized_taxonomy_code'])

In [13]:
data.shape

(1515, 47)

In [27]:
def preprocess_for_prediction(dataframe):
    seq = dataframe.groupby('student_id').apply(
        lambda r: (
            r['factorized_taxonomy_code'].values[:]
        )
    )
    dataset = tf.data.Dataset.from_generator(
        generator=lambda: seq,
        output_types=(tf.int32)
    )
    dataset = dataset.map(
        lambda factorized_taxonomy_code: (
            tf.one_hot(factorized_taxonomy_code, depth=700)
        )
    )
    dataset = dataset.padded_batch(
        batch_size=64,
        padding_values=(tf.constant(-1, dtype=tf.float32)),
        padded_shapes=([44,None])
    )
    return dataset

In [28]:
dataset = preprocess_for_prediction(data)

In [29]:
preds = model.predict(dataset)

In [17]:
len(preds)

225

In [18]:
preds[0]

array([[0.50010425, 0.50384927, 0.500363  , ..., 0.4970426 , 0.49445853,
        0.508408  ],
       [0.49629036, 0.5047769 , 0.4988176 , ..., 0.49531838, 0.49282914,
        0.51038986],
       [0.4931501 , 0.508169  , 0.50035304, ..., 0.49001163, 0.4915433 ,
        0.51212895],
       ...,
       [0.44282502, 0.54463005, 0.49871698, ..., 0.43799603, 0.45592615,
        0.54973525],
       [0.44282502, 0.54463005, 0.49871698, ..., 0.43799603, 0.45592615,
        0.54973525],
       [0.44282502, 0.54463005, 0.49871698, ..., 0.43799603, 0.45592615,
        0.54973525]], dtype=float32)

In [19]:
len(preds[0])

44